In [2]:
import pytest
from pathlib import Path
from pyqmrlab.t1 import VFA
import httplib2
import shutil
import numpy as np
import scipy as sp


In [7]:
class TestCore(object):
    def setup(self):
        self.tmpPath = Path("tmp/")

    @classmethod
    def teardown_class(cls):
        tmpPath = Path("tmp/")
        shutil.rmtree(tmpPath)
        pass

    # --------------attribute tests-------------- #
    def test_data_url_link_exists(self):

        vfa_obj = VFA()

        h = httplib2.Http()

        try:
            resp = h.request(vfa_obj.data_url, "HEAD")
            assert int(resp[0]["status"]) < 400
        except Exception:
            pytest.fail("Website not found.")

    # --------------download tests-------------- #
    def test_download(self):
        vfa_obj = VFA()
        vfa_obj.download(self.tmpPath)

        expected_folder = self.tmpPath / "vfa_t1"
        expected_files = [
            self.tmpPath / "vfa_t1/VFAData.nii.gz",
            self.tmpPath / "vfa_t1/B1map.nii.gz",
            self.tmpPath / "vfa_t1/Mask.nii.gz",
        ]
        assert expected_folder.is_dir()
        for file in expected_files:
            assert file.is_file()

    # --------------load tests-------------- #
    def test_load(self):
        vfa_obj = VFA()

        VFAData = self.tmpPath / "vfa_t1/VFAData.nii.gz"
        B1map = self.tmpPath / "vfa_t1/B1map.nii.gz"
        Mask = self.tmpPath / "vfa_t1/Mask.nii.gz"

        vfa_obj.load(VFAData, B1map, Mask)

        assert isinstance(vfa_obj.VFAData, np.ndarray)
        assert isinstance(vfa_obj.B1map, np.ndarray)
        assert isinstance(vfa_obj.Mask, np.ndarray)

        expected_shape_data = (128, 128, 1, 2)
        expected_shape_b1 = (128, 128)
        expected_shape_mask = (128, 128)

        assert vfa_obj.VFAData.shape == expected_shape_data
        assert vfa_obj.B1map.shape == expected_shape_b1
        assert vfa_obj.Mask.shape == expected_shape_mask

    # --------------simulate tests-------------- #
    def test_simulate(self):
        vfa_obj = VFA()
        params = {"flip_angle": [3, 20], "repetition_time": 0.015, "T1": 0.850}

        Mz = VFA.simulate(params, "analytical")

        expected_value = np.array([0.04859526, 0.07795592])
        actual_value = Mz

        assert np.allclose(actual_value, expected_value)

    # --------------fit tests-------------- #
    def test_fit_simulate_1vox(self):
        vfa_obj = VFA()
        params = {"flip_angle": [3, 20], "repetition_time": 0.015, "T1": 0.850}

        Mz = VFA.simulate(params, "analytical")

        vfa_obj.VFAData = np.ones((1, 1, 1, 2))

        vfa_obj.VFAData[0, 0, 0, :] = Mz

        vfa_obj.B1map = np.ones((1, 1))
        vfa_obj.Mask = np.ones((1, 1))

        vfa_obj.fit()

        expected_value = params["T1"]
        actual_value = vfa_obj.T1

        assert actual_value == pytest.approx(expected_value, abs=0.01)

    def test_fit_simulate_3vox(self):
        vfa_obj = VFA()
        params = {"flip_angle": [3, 20], "repetition_time": 0.015, "T1": 0.850}
        vfa_obj.VFAData = np.ones((3, 1, 1, 2))
        vfa_obj.Mask = np.ones((3, 1))
        vfa_obj.B1map = np.ones((3, 1))
        vfa_obj.B1map[1, 0] = 0.95
        vfa_obj.B1map[2, 0] = 1.05

        Mz = VFA.simulate(params, "analytical")
        vfa_obj.VFAData[0, 0, 0, :] = Mz

        params["flip_angle"] = np.array([3, 20]) * vfa_obj.B1map[1, 0]
        Mz = VFA.simulate(params, "analytical")
        vfa_obj.VFAData[1, 0, 0, :] = Mz

        params["flip_angle"] = np.array([3, 20]) * vfa_obj.B1map[2, 0]
        Mz = VFA.simulate(params, "analytical")
        vfa_obj.VFAData[2, 0, 0, :] = Mz

        vfa_obj.fit()

        expected_value = np.array([params["T1"], params["T1"], params["T1"]])
        actual_value = vfa_obj.T1

        assert np.allclose(actual_value, expected_value)

    def test_fit(self):
        vfa_obj = VFA()

        VFAData = self.tmpPath / "vfa_t1/VFAData.nii.gz"
        B1map = self.tmpPath / "vfa_t1/B1map.nii.gz"
        Mask = self.tmpPath / "vfa_t1/Mask.nii.gz"

        vfa_obj.load(VFAData, B1map, Mask)

        vfa_obj.fit()

        expected_median_value = 1.33
        actual_median_value = np.median(vfa_obj.T1[vfa_obj.T1 != 0])

        assert actual_median_value == pytest.approx(expected_median_value, abs=0.1)

    # --------------save tests-------------- #
    def test_save(self):
        vfa_obj = VFA()

        VFAData = self.tmpPath / "vfa_t1/VFAData.nii.gz"
        B1map = self.tmpPath / "vfa_t1/B1map.nii.gz"
        Mask = self.tmpPath / "vfa_t1/Mask.nii.gz"

        vfa_obj.load(VFAData, B1map, Mask)

        vfa_obj.fit()

        outputFile = self.tmpPath / "T1.nii.gz"
        vfa_obj.save(outputFile)

        assert outputFile.is_file()

In [8]:
test_save()

NameError: name 'test_save' is not defined